Automatic Update/Collect Draws

1. Request 50 Draws each time of request (max_permitted)
2. Check the response
3. Save it in json()
4. Parsing and store the data
5. Check the Update Interval (date?) -> Request the new draw at the right time.
6. ...

### Why drawrange over daterange?
drawrange allows upto 50 at once while daterange allows 90 days.

Since the lottery is weekly-drawn, daterange allows to get maximum 12 draws per request.

### About MonWedLotto?
MonWedLotto use the same machine, and draws twice weekly; mon and wed.

Since the purpose of this project is to analyze the tendency of a "machine" I did not separated two draws. 

Even draw Nums are Monday

Odd  draw Nums are Wed

### Why 2012 Jan?
1. API supports draws starting from different date.
2. Apply fairly "recent" tendencies.

In [325]:
import requests
import pandas as pd
from enum import Enum
import time

In [356]:
# Classes - Different Lottery has diff rules

class Lotto:
    def __init__(self, game_name, next_draw_num):
        self.game_name = game_name
        self.next_draw_num = next_draw_num
        self.interval = 7
        self.last_draw = 0

In [357]:
# Basic Parameter Setup
# Two Lotto shares the same data.api url as following
url = 'https://data.api.thelott.com/sales/vmax/web/data/lotto/results/search/drawrange'

# Payload Setup 
payload_template = {'MinDrawNo': None, 'MaxDrawNo': None, 'Product': None, 'CompanyFilter': ["NSWLotteries"]}

In [358]:
# Create request payload in json format (dict)
# Request 50 draws at one time (api request limit)
def create_payload(game: Lotto) -> dict:
    ret_payload = payload_template.copy()
    ret_payload['MinDrawNo'] = game.draw_num
    ret_payload['MaxDrawNo'] = game.draw_num + 49
    ret_payload['Product'] = game.game_name
    return ret_payload

In [359]:
# request Setup
wed_data = pd.DataFrame()
sat_data = pd.DataFrame()

def do_request(url, payload):
    try:
        r = requests.post(url, json = payload, timeout = (2,5))
    except Timeout:
        print('Request TimeOut')
        return
    else:
        if r.status_code == 200:
            print("Request Success")
        else:
            print("Request Failed - ", r.status_code)
            return
    
    response = r.json()
    
    if response['ErrorInfo'] != None:
        print("Error - May have issues with parameter of payload")
        print(payload)
        return
    
    return response

def interpret_response(data, response):
    draws = pd.DataFrame.from_dict(response['Draws'])
    try:
        data = data.append(draws)
    except:
        print("parse_response: Data Append Failed")
        return
    else:
        print("parse_response: Data Append Success")
        return data, len(draws)

In [390]:
# Website has no previous draws for MonWedLotto
startNo_monWed = 3108 # 2012 Jan 30
startNo_sat = 3185 # 2012 Jan 28
wed_game = Lotto('MonWedLotto', 4055)
sat_game = Lotto('TattsLotto', startNo_sat)

In [391]:
# Script Function

def crawl_all(wed_game: Lotto, sat_game: Lotto):
    wed_data = pd.DataFrame()
    sat_data = pd.DataFrame()
    
    wed_crawl_done = False
    while not wed_crawl_done:
        wed_payload = create_payload(wed_game)
        response = do_request(url, wed_payload)
        if len(response['Draws']) == 0 or response == None:
            wed_crawl_done = True
        else:
            wed_data, num_draws = interpret_response(wed_data, response)
            print("MonWed Draws from: ", wed_game.draw_num, "to", wed_game.draw_num + num_draws - 1)
            wed_game.draw_num += num_draws
            time.sleep(2)
            
    print("monWed Draws Completed upto", wed_game.draw_num - 1)
    print("Now Moving on to Sat Draws")
    
    sat_crawl_done = False
    while not sat_crawl_done:
        sat_payload = create_payload(wed_game)
        response = do_request(url, sat_payload)
        if  len(response['Draws']) == 0 or response == None:
            sat_crawl_done = True
        else:
            sat_data, num_draws = interpret_response(sat_data, response)
            print("Sat Draws from: ", sat_game.draw_num, "to", sat_game.draw_num + num_draws - 1)
            sat_game.draw_num += num_draws
            time.sleep(2)            
    
    print("sat Draws Completed upto", sat_game.draw_num - 1)
    print("Now Wrapping up crawlling")
    
    
    return wed_data, sat_data, wed_game, sat_game

In [392]:
wed_data, sat_data, wed_game, sat_game = crawl_all(wed_game,sat_game)

Request Success
parse_response: Data Append Success
MonWed Draws from:  4055 to 4061
Request Success
Request Success


In [393]:
wed_data

,ProductId,DrawNumber,DrawDate,PrimaryNumbers,SecondaryNumbers,TicketNumbers,Dividends
0,MonWedLotto,4061,2021-03-17T00:00:00,"[38, 22, 35, 24, 7, 4]","[31, 1]",None,"[{'Division': 1, 'BlocNumberOfWinners': 1, 'Bl..."
1,MonWedLotto,4060,2021-03-15T00:00:00,"[21, 45, 44, 24, 1, 16]","[29, 36]",None,"[{'Division': 1, 'BlocNumberOfWinners': 0, 'Bl..."
2,MonWedLotto,4059,2021-03-10T00:00:00,"[6, 42, 26, 30, 8, 25]","[36, 31]",None,"[{'Division': 1, 'BlocNumberOfWinners': 0, 'Bl..."
3,MonWedLotto,4058,2021-03-08T00:00:00,"[9, 27, 6, 43, 21, 19]","[24, 10]",None,"[{'Division': 1, 'BlocNumberOfWinners': 0, 'Bl..."
4,MonWedLotto,4057,2021-03-03T00:00:00,"[24, 42, 7, 40, 36, 25]","[44, 15]",None,"[{'Division': 1, 'BlocNumberOfWinners': 0, 'Bl..."
5,MonWedLotto,4056,2021-03-01T00:00:00,"[11, 10, 45, 44, 21, 20]","[5, 24]",None,"[{'Division': 1, 'BlocNumberOfWinners': 1, 'Bl..."
6,MonWedLotto,4055,2021-02-24T00:00:00,"[45, 44, 40, 17, 16, 29]","[13, 21]",None,"[{'Division': 1, 'BlocNumberOfWinners': 0, 'Bl..."


In [296]:
recent = wed_data['DrawDate'][0]
a = pd.to_datetime(recent)

In [297]:
a + pd.DateOffset(7)

Timestamp('2001-12-12 09:01:00')

In [369]:
testing = {"Draws":[],"ErrorInfo": None,"Success":True}

In [384]:
len(testing['Draws'])

0

In [307]:
r = requests.post(url, json = json)
json_response = r.json()
repository = json_response

In [308]:
repository

{'Draws': None,
 'ErrorInfo': {'SystemId': 200,
  'ErrorNo': 54006,
  'DisplayMessage': 'Date range exceeds the 90 days limit.',
  'ContactSupport': False,
  'SupportErrorReference': '200-54006'},
 'Success': False}

In [170]:
len(repository['ErrorInfo'])

5